# Scraping articles from Il Post website

In [2]:
#I import the necessary packages to webscrape
import requests
from lxml.html import fromstring
import time
import pandas as pd

#I create a function to extract the information from the website
def fetch_article_html(url):
    response = requests.get(url)
    return response.text

#I create a function to extract the text of the article and to put together all of the paragraphs
def extract_article_text(html):
    tree = fromstring(html)
    article_text = tree.xpath("//div[2]/p/text() | //div[2]/p/a/text()")
    return ' '.join(article_text)

#I define the url of the website and loop over the 6 pages present in the section I am interested in
MAXPAGES = 6
baseurl = "https://www.ilpost.it/tag/elezioni-2022/page/"
allurls = [f"{baseurl}{i}/" for i in range(1, MAXPAGES + 1)]

#I create a list in which to store the information
articles_data = []

#I loop over every url and check if the page exist. Then I extract the link and title of each article from the website and 
#then for every link (so for every aerticle) I retrieve the test of that article. 
for url in allurls:
    try:
        #I check if the page exist and I fetch the HTML source from the website
        response = requests.get(url)
        if response.status_code == 404:
            print("Page not found:", url)
            continue
        htmlsource = response.text

        tree = fromstring(htmlsource)

        #I find the links and titles
        article_links = tree.xpath("//div/h2/a/@href")
        article_title = tree.xpath("//div/h2/a/text()")

        for link, title in zip(article_links, article_title):
            try:
                #I fetch the HTML source of each article page
                article_html = fetch_article_html(link)

                #I extract the full article text
                article_text = extract_article_text(article_html)

                #I append the links, titles and text to the list created above
                articles_data.append({
                    'Link': link,
                    'Title': title,
                    'Text': article_text
                })

                #I delay of one second before the next request
                time.sleep(1)  

            except requests.exceptions.RequestException as e:
                print("Connection error:", e)
                continue

    except requests.exceptions.RequestException as e:
        print("Connection error:", e)
        continue

#I create a dataframe from the articles data dictionary, I add anew column with the name of the news outlet and I save 
#the file to a csv file
articles_IlPost = pd.DataFrame(articles_data)
articles_IlPost['News outlet'] = pd.Series('Il Post', index=articles_IlPost.index)
articles_IlPost.to_csv('articles_IlPost.csv', index=False)

#I print the DataFrame
articles_IlPost

,Link,Title,Text,News outlet
0,https://www.ilpost.it/2022/09/27/nuovo-parlame...,Come sarà il prossimo parlamento,Il parlamento emerso dalle elezioni politiche ...,Il Post
1,https://www.ilpost.it/2022/10/30/inserzioni-po...,Piazzare spot politici sui social è sempre più...,Questo non vuol dire che i politici abbiano sm...,Il Post
2,https://www.ilpost.it/2022/10/21/consultazioni...,Che tempi ci sono per l’insediamento del governo,Venerdì Giorgia Meloni ha ricevuto e accettat...,Il Post
3,https://www.ilpost.it/2022/10/20/consultazioni...,È finita la prima giornata di consultazioni,Si è conclusa al Quirinale la prima delle due ...,Il Post
4,https://www.ilpost.it/2022/10/19/calendario-co...,Il calendario delle consultazioni di Mattarella,Dopo l’elezione dei presidenti di Camera e Sen...,Il Post
...,...,...,...,...
149,https://www.ilpost.it/2022/07/31/rosatellum-fa...,Perché la legge elettorale “favorisce le allea...,"Il “Rosatellum” , la legge elettorale con cui...",Il Post
150,https://www.ilpost.it/2022/07/30/azione-carfag...,Azione si allarga,In un evento organizzato alla sala della stamp...,Il Post
151,https://www.ilpost.it/2022/07/29/paola-taverna...,Le urla di Paola Taverna in Senato,Dato che il Movimento 5 Stelle (M5S) ha confe...,Il Post
152,https://www.ilpost.it/2022/07/29/meloni-salvin...,Nella destra i rapporti con la Russia sono div...,Negli ultimi due giorni alcune notizie e retro...,Il Post


I create a condition with a list of rows and then use it to extract those rows from the dataset and put into another dataset, 
which will serve for the labeled dataset. Then, I drop those columns from the original dataset, as this is the unlabeled dataset. I save both datasets and reset their indexes. 

In [8]:
condition = list(range(1, 15)) + list(range(74, 89)) + list(range(141, 154))
articlesPost_labeled = articles_IlPost.loc[condition].copy()
articlesPost_labeled.head()
articlesPost_labeled.to_csv('articlesPost_labeled.csv', index=False)
articlesPost_unlabeled = articles_IlPost.drop(list(range(1, 15)) + list(range(74, 89)) + list(range(141, 154)))
articlesPost_unlabeled.to_csv('articlesPost_unlabeled.csv', index=False)
#I restore the indexes of both dataframes
articlesPost_labeled.reset_index(drop=True, inplace=True)
articlesPost_unlabeled.reset_index(drop=True, inplace=True)

In [9]:
#I check that everything went smoothly
articlesPost_unlabeled

,Link,Title,Text,News outlet
0,https://www.ilpost.it/2022/09/27/nuovo-parlame...,Come sarà il prossimo parlamento,Il parlamento emerso dalle elezioni politiche ...,Il Post
1,https://www.ilpost.it/2022/09/28/analisi-fluss...,Cosa ci dicono le analisi sui flussi elettorali,"Come dopo ogni elezione, anche negli ultimi gi...",Il Post
2,https://www.ilpost.it/2022/09/28/fratelli-ital...,Fratelli d’Italia non ha vinto grazie al voto ...,Negli ultimi giorni sui social network si è di...,Il Post
3,https://www.ilpost.it/2022/09/27/astensione-vi...,I partiti perdono montagne di voti,Il modo con cui vengono solitamente presentati...,Il Post
4,https://www.ilpost.it/2022/09/26/elezioni-poli...,La destra avrà un’ampia maggioranza in parlamento,La coalizione di destra ha vinto le elezioni p...,Il Post
...,...,...,...,...
107,https://www.ilpost.it/2022/08/05/agenda-draghi/,Non esiste davvero una “agenda Draghi”,Nelle prime settimane di campagna elettorale i...,Il Post
108,https://www.ilpost.it/2022/08/04/bonelli-frato...,Cosa vogliono Angelo Bonelli e Nicola Fratoianni,L’accordo fra Partito Democratico e Azione ra...,Il Post
109,https://www.ilpost.it/2022/08/03/diritto-di-tr...,Cos’è il “diritto di tribuna”,"Martedì, poco dopo che il segretario del Parti...",Il Post
110,https://www.ilpost.it/2022/08/03/italia-viva-e...,Italia Viva è rimasta da sola,"L’accordo fra Azione e Partito Democratico, ch...",Il Post


In [11]:
#I check that everything went smoothly
articlesPost_labeled

,Link,Title,Text,News outlet
0,https://www.ilpost.it/2022/10/30/inserzioni-po...,Piazzare spot politici sui social è sempre più...,Questo non vuol dire che i politici abbiano sm...,Il Post
1,https://www.ilpost.it/2022/10/21/consultazioni...,Che tempi ci sono per l’insediamento del governo,Venerdì Giorgia Meloni ha ricevuto e accettat...,Il Post
2,https://www.ilpost.it/2022/10/20/consultazioni...,È finita la prima giornata di consultazioni,Si è conclusa al Quirinale la prima delle due ...,Il Post
3,https://www.ilpost.it/2022/10/19/calendario-co...,Il calendario delle consultazioni di Mattarella,Dopo l’elezione dei presidenti di Camera e Sen...,Il Post
4,https://www.ilpost.it/2022/10/18/capigruppo-pa...,I partiti hanno eletto i propri capigruppo in ...,Il Post è una testata registrata presso il Tri...,Il Post
5,https://www.ilpost.it/2022/10/16/nuovo-governo...,Cosa succede ora per arrivare al nuovo governo,"Già prima delle elezioni del 25 settembre, vis...",Il Post
6,https://www.ilpost.it/2022/10/15/trattative-nu...,Come vanno le trattative per il nuovo governo,Trascorse quasi tre settimane dalla vittoria a...,Il Post
7,https://www.ilpost.it/2022/10/08/conteggio-leg...,Molti candidati non sanno ancora se sono stati...,La Cassazione ha finito di controllare i vot...,Il Post
8,https://www.ilpost.it/2022/10/03/bossi-salvini...,Un pezzo della Lega vuole tornare Lega Nord,Il cattivo risultato della Lega alle elezioni...,Il Post
9,https://www.ilpost.it/2022/10/01/destra-moratt...,Il primo problema della nuova maggioranza è in...,Meno di una settimana dopo le elezioni la coal...,Il Post
